In [ ]:
import re
import io
import os
import time
import requests
import numpy as np
import pandas as pd
from PIL import Image

# Definir las diferentes constantes del programa

In [ ]:
DOG_API   = "https://random.dog/woof.json"
DUCK_API  = "https://random-d.uk/api/random"
FOX_API   = "https://randomfox.ca/floof/"
BASE_RAW_DIRECTORY = f'data/raw'
BASE_MASTER_DIRECTORY = f'data/master'
NUMBER_IMAGES = 5

# Definición de las funciones a utilizar

In [ ]:
def processImage(base_dir: str, byte_string: str, animal: str, index: int) -> None:  
    """
    Función que permite guardar las imagenes en un directorio
    """
    os.makedirs(f'{base_dir}/{animal}', exist_ok=True) # Permite generar el directorio si este no existe
    with open(f'{base_dir}/{animal}/{index}.png', 'wb') as f:
        f.write(byte_string)

# Descarga de imagenes

In [ ]:
# Descarga de imagenes de perros
for index in range(NUMBER_IMAGES):
    dog_data = requests.get(DOG_API)
    if dog_data.status_code == 200:
        processImage(
            BASE_RAW_DIRECTORY,
            requests.get(
              dog_data.json()["url"]
            ).content,
            'dog',
            index
        )
    else:
        break
    time.sleep(3)

In [ ]:
# Descarga de imagenes de patos
for index in range(NUMBER_IMAGES):
    duck_data = requests.get(DUCK_API)
    if duck_data.status_code == 200:
        processImage(
            BASE_RAW_DIRECTORY,
            requests.get(
              duck_data.json()["url"]
            ).content,
            'duck',
            index
        )
    else:
        break
    time.sleep(3)

In [ ]:
# Descarga de imagenes de zorros
for index in range(NUMBER_IMAGES):
    fox_data = requests.get(FOX_API)
    if fox_data.status_code == 200:
        processImage(
            BASE_RAW_DIRECTORY,
            requests.get(
              fox_data.json()["image"]
            ).content,
            'fox',
            index
        )
    else:
        break
    time.sleep(3)

# Estandarización de tamaños
Se busca que todas las imagenes tengan una dimensión 100 x 100

In [ ]:
final_data = [[],[]] #La primer entrada será para guardar las categorias, la segunda permitirá almacenar la representación numérica de la imagen
for animal in os.listdir(BASE_RAW_DIRECTORY):
    for idx, picture in enumerate(os.listdir(f'{BASE_RAW_DIRECTORY}/{animal}')):
        try:
            if re.match(r'([0-9])+.png$', picture):
                img = Image.open(f'{BASE_RAW_DIRECTORY}/{animal}/{picture}').convert('RGB') #Convertir a RGB, para eliminar canales como el Alfa que tienen algunas imagenes
                image_resized = img.resize((100, 100))
                img_byte_array = io.BytesIO() # Generar un stream virtual para obtener la cadena de byte-string
                image_resized.save(img_byte_array, format='PNG') # Guardar en el stream virtual
                img_bytes = img_byte_array.getvalue() # Obtener el byte-string
                processImage(
                    BASE_MASTER_DIRECTORY,
                    img_bytes,
                    animal,
                    idx
                )
                final_data[0].append(animal)
                final_data[1].append(np.array(image_resized).flatten()) # .flatten permite redimensionar la imagen a una sola dimensión
        except:
            print(f'{animal}/{picture} corrupted')

In [ ]:
df = pd.DataFrame(final_data[1]) # Generar el dataframe de la representación numerica de las imagenes
df = df / 255.0 # Estandarización de los pixeles

In [ ]:
df.columns = [f'img_{col}' for col in df.columns] # Cambiar el nombre de las columnas

In [ ]:
df_categories = pd.DataFrame(final_data[0], columns = ["animal"]) # Generando el dataframe de las categorias

In [ ]:
df_final = pd.concat([df, df_categories], axis = 1) # Uniendo ambos dataframes

In [ ]:
df_final.to_csv("data/ADT.csv", index = False)